In [12]:
import requests 
url = "https://www.nownews.com/cat/column/"

headers= {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url)
# print(res.text)

In [11]:
from bs4 import BeautifulSoup as bs
soup = bs(res.text,"lxml")
# print(soup)

In [10]:
### 抓取 1 ~ 10 篇文章連結
links = []

# 抓取 1~5
for ele in soup.select("div.sliderBlk a"):
#     print(ele)
    url = ele["href"]
#     print(url)
    links.append(url)
    
# 抓取 6~10
for ele in soup.select("div.leftCol div.listBlk a"):
#     print(ele)
    links.append(ele["href"])
    
for url in links:
    print(url)

https://www.nownews.com/news/5424941
https://www.nownews.com/news/5424915
https://www.nownews.com/news/5424737
https://www.nownews.com/news/5424908
https://www.nownews.com/news/5424900
https://www.nownews.com/news/5424663
https://www.nownews.com/news/5424901
https://www.nownews.com/news/5424615
https://www.nownews.com/news/5424853
https://www.nownews.com/news/5424811
